In [ ]:
require(data.table)
require(ggplot2)
require(reshape2)

data_dir="/oak/stanford/groups/smontgom/erobb/data"

files = c("ESN.SVs.11.5.txt",
        "GWD.SVs.13.6.txt",
        "LWK.SVs.10.4.txt",
        "MKK.SVs.14.4.txt",
        "YRI.SVs.8.2.txt",
        "MSL.SVs.11.4.txt")
names(files) <- c("ESN", "GWD", "LWK", "MKK", "YRI", "MSL")

#pops = c("ESN", "GWD", "LWK", "MKK", "YRI", "MSL")
pop = "ESN"

# Read in deseq2 expression data
expr_file=sprintf('%s/deseq/deseq2ScaledGeneCounts.%s.bed',data_dir,pop)
expr = read.delim(expr_file)
row.names(expr) = expr[,"ID"] # set row name to gene ID

# Read in covariates matrix
covs_file = sprintf("/oak/stanford/groups/smontgom/mdegorte/durga/africa/fastqtl/SVs/%s/%s",pop,files[[pop]])
covs = read.table(covs_file, header = T, sep = '\t', row.names = 1)

# Read in top eQTL
eqtl_file=sprintf('%s/eqtls/sorted.byGene.dist.hwe.af.%s.top1.eQTL.nominal.hg38a.txt', data_dir, pop)
eqtl_df = read.table(eqtl_file, header = T, row.names = 'feature')
# Read in vcf data for each top eQTL
vcf_file = sprintf("%s/eqtls/AF.all.%s.hg38aID.eQTLs.ba.vcf", data_dir, pop)
vcf_df = read.table(vcf_file, header=T, skip=0, row.names="GENE")

# Filter to IDs with available covariates
id_keep = colnames(expr)[5:length(colnames(expr))]
id_keep = intersect(id_keep,colnames(covs))
covs = covs[,id_keep]
expr = expr[,id_keep]

# 1 if alt allele is present, 0 otherwise
vcf_df[,id_keep] = sapply(vcf_df[,id_keep] == '0|0', as.numeric)
vcf_df = vcf_df[,id_keep]

# Scale to mean 0 variance 1 (over each gene)
expr = scale(t(expr))

# # For each gene in the expression file, perform a linear regression 
resids = matrix(, ncol = ncol(expr), nrow = nrow(expr))
rownames(resids) = rownames(expr)
colnames(resids) = colnames(expr)

for(i in 1:ncol(expr)){ # for each gene! seems slow but not too bad
    gene = colnames(expr)[i]
    if (!all(is.na(vcf_df[gene,]))) {
        gcovs = rbind(covs, vcf_df[1,])
    } else {
        gcovs = covs
    }
    data = as.data.frame(cbind(expr[, i], t(gcovs)))
    colnames(data) = c('deseq2ScaledGeneCounts', rownames(gcovs))
    model = lm(deseq2ScaledGeneCounts ~ ., data = data)
    resids[, i] = model$residuals
}
# Center and scale, then transpose
resids = t(scale(resids))

# Write out the residuals
out_file = sprintf('%s/eoutliers/%s_exprResiduals.tsv', data_dir, pop)
write.table(matrix(c('Id', colnames(resids)), nrow = 1), out_file, quote = F, row.names = F, col.names = F, sep = '\t')
write.table(resids, out_file, row.names = T, col.names = F, quote = F, sep = '\t', append = T)



Loading required package: data.table

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘data.table’”
Loading required package: reshape2

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘reshape2’”


In [ ]:
resids

In [32]:
install.packages("ggplot2")
library(ggplot2)

pop="ESN"
r_file = sprintf('%s/eoutliers/%s_exprResiduals.tsv', data_dir, pop)

# Write out the residuals
resids = read.table(r_file, header=T, row.names=1)
resids = data.frame(t(resids))

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [35]:
# ggplot(resids, aes(x=ENSG00000227232.5)) +  geom_histogram()
# ggplot(resids, aes(x=ENSG00000279457.4)) +  geom_histogram()

In [38]:
model$residuals

HG02941     HG02976     HG03114     HG03135     HG03189     HG03271 
 1.28620986  0.09517781 -0.56483295 -0.98152323 -0.86660353  0.27340020 
    HG03313     HG03367     HG02943     HG02977     HG03115     HG03136 
 1.15570805  1.84581228 -0.60016152  0.36022590  1.24972308  0.57575629 
    HG03190     HG03279     HG03280     HG03342     HG03369     HG02981 
 0.79866625 -0.66281871  1.45999878 -1.38466743 -0.81170084  0.56394649 
    HG03117     HG03139     HG03291     HG03343     HG03370     HG02946 
-1.00044060  0.66985292 -0.87781413  0.16201296  1.03768584 -0.77909504 
    HG03118     HG03157     HG03195     HG03294     HG03351     HG03372 
 1.00567383  0.73618760 -1.06354636 -0.22085854 -0.47299408 -0.78916378 
    HG02947     HG03099     HG03120     HG03159     HG03196     HG03295 
 0.65458988 -0.77265545  0.55041155 -1.22616954  1.07295356  1.07713672 
    HG03352     HG03499     HG02952     HG03100     HG03121     HG03124 
 0.05362686 -0.93626991 -1.20449322  0.95033290 -1.24160174  0.52823887 
    HG03160     HG03198     HG03297     HG03354     HG02953     HG03103 
 0.70473047 -0.88713051 -0.27888672 -0.11081244  0.73880210  0.69847599 
    HG03123     HG03126     HG03162     HG03199     HG03514     HG02968 
-1.16993239 -1.17596771 -1.57891712  0.79739466 -0.76423428  0.39662295 
    HG03105     HG03127     HG03163     HG03202     HG03300     HG03515 
-0.85623342  1.22084605  0.10009353  0.67680690 -1.04745196  0.56155414 
    HG02970     HG03108     HG03129     HG03166     HG03265     HG03301 
-0.79832001 -1.22107957 -0.91202453  0.10455327  0.48745561  0.93865427 
    HG03517     HG02922     HG02971     HG03109     HG03130     HG03168 
-1.78923990 -0.99170721  0.68419375  1.57894213  0.91873240 -1.42367574 
    HG03169     HG03267     HG03303     HG03518     HG02923     HG02973 
 0.41521506 -0.84946944 -0.86530409  0.95282555  0.81888591 -0.66999162 
    HG03111     HG03172     HG03268     HG03304     HG03363     HG02938 
-0.54823234  0.78963466  0.21466402  0.82897612 -1.29769825  1.07845967 
    HG02974     HG03112     HG03133     HG02944     HG03132     HG03193 
-0.62274959  0.78715632  0.62820675  0.79319312 -1.17635441  0.71797129 
    HG03298     HG03520     HG03175     HG03270     HG03311     HG03366 
 0.77771425 -1.27982232  0.78505669 -0.97097687  0.67840878 -0.32284025 
    HG03521 
 1.02890838

### Label outliers

In [36]:
quantile(resids)

ERROR: Error in quantile.default(resids): missing values and NaN's not allowed if 'na.rm' is FALSE


In [150]:
## Pick outliers using median Z-score >= 2
## Remove individuals with >= 50 outliers 
## Write to file
## Also write out list of individuals that pass outlier count threshold
## Also write out the most extreme individual per gene without threshold (after removing individuals with >= 50 outliers)
z_thresh = 3
# AFGR individuals have more on average, so we can choose a higher threshold than Watershed
z_ind_filt = 75

pops = c("ESN", "GWD", "LWK", "MKK", "YRI", "MSL")
for (pop in pops) {
    r_file = sprintf('%s/%s_exprResiduals.tsv', data_dir, pop)

    # Write out the residuals
    resids = read.table(r_file, header=T, row.names=1)
    resids = data.frame(t(resids))
    
    w = abs(resids) > z_thresh
    w = data.frame(sapply(data.frame(w), function(x) as.integer(x)))
    rownames(w) = rownames(resids)
    indiv_noutliers = data.frame(rowSums(w))
    indiv_keep = rownames(indiv_noutliers)[which(indiv_noutliers < z_ind_filt)]

    mean(indiv_noutliers[,])
    sqrt(var(indiv_noutliers[,]))
    median(indiv_noutliers[,])
    print(sprintf("Keeping %i / %i individuals with z_thresh = %i, max outliers = %i",length(indiv_keep), length(rownames(w)), z_thresh, z_ind_filt))
    write.table(indiv_keep, sprintf('%s/ids_outlier_filtered_%s_t%if%i.txt', data_dir, pop, z_thresh, z_ind_filt), quote = F, sep = '\t', col.names = F, row.names = F)
    write.table(w, sprintf('%s/eOutlier_scores_%s_t%i.txt', data_dir, pop, z_thresh))
}


[1] "Keeping 62 / 97 individuals with z_thresh = 3, max outliers = 75"
[1] "Keeping 72 / 112 individuals with z_thresh = 3, max outliers = 75"
[1] "Keeping 60 / 96 individuals with z_thresh = 3, max outliers = 75"
[1] "Keeping 102 / 164 individuals with z_thresh = 3, max outliers = 75"
[1] "Keeping 32 / 41 individuals with z_thresh = 3, max outliers = 75"
[1] "Keeping 52 / 83 individuals with z_thresh = 3, max outliers = 75"


In [164]:
# Create a combined file for AFR
pops = c("ESN", "GWD", "LWK", "MKK", "YRI", "MSL")
afr_id_filt = data.frame()
afr_eout_scores = data.frame()
for (pop in pops) {
    afr_id_filt = rbind(afr_id_filt,
          read.table(sprintf('%s/ids_outlier_filtered_%s_t%if%i.txt', data_dir, pop, z_thresh, z_ind_filt), sep=" "))
    afr_eout_scores = rbind(afr_eout_scores,
          read.table(sprintf('%s/eOutlier_scores_%s_t%i.txt', data_dir, pop, z_thresh), sep=" "))
}
write.table(afr_id_filt, sprintf('%s/ids_outlier_filtered_AFR_t%if%i.txt', data_dir, z_thresh, z_ind_filt), quote = F, sep = '\t', col.names = F, row.names = F)
write.table(afr_eout_scores, sprintf('%s/eOutlier_scores_AFR_t%i.txt', data_dir, z_thresh))


In [166]:
 read.table(sprintf('%s/ids_outlier_filtered_%s_t%if%i.txt', data_dir, 'AFR', z_thresh, z_ind_filt), sep=" ")

V1
<chr>
HG02941
HG02976
HG03114
HG03135
HG03189
HG03271
HG02943
HG02977
HG03115


In [165]:
read.table(sprintf('%s/eOutlier_scores_%s_t%i.txt', data_dir, 'AFR', z_thresh), sep=" ")

,ENSG00000227232.5,ENSG00000279457.4,ENSG00000230021.9,ENSG00000225972.1,ENSG00000225630.1,ENSG00000237973.1,ENSG00000229344.1,ENSG00000248527.1,ENSG00000198744.5,ENSG00000237491.8,⋯,ENSG00000067048.16,ENSG00000183878.15,ENSG00000215414.4,ENSG00000154620.5,ENSG00000241859.6,ENSG00000165246.13,ENSG00000176728.7,ENSG00000131002.11,ENSG00000012817.15,ENSG00000198692.9
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
HG02941,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
HG02976,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
HG03114,0,0,0,0,0,0,0,0,0,0,⋯,0,0,1,0,0,0,0,0,0,0
HG03135,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
HG03189,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
HG03271,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
HG03313,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
HG03367,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
HG02943,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
